# Data Science Moringa Group Project - A Data Science Phase 3 Project
# Final Project Submission
#####:

Student name: Pete Njagi,
          

Student pace: DS-full time-Online

Scheduled project review date/time: March 10th 2024

###GitHub Link:
https://github.com/petezdj/Phase3Project

<img src="big%20data-1.jpg" alt="drawing" width="700"/>

#### Important Project Files:

1. index.ipynb (Main Juypter document)
2. Presentation.pdf (Presentation)
3. Data_sci_jobs.csv (Main Data)
4. Optional Project Pdf

## Introduction
KenyaData Insights, 

In [2]:
import pandas as pd

In [3]:
# We must start with loading the datasets
train_values = pd.read_csv('TrainingSetValues.csv')
train_labels = pd.read_csv('TrainingSetLabels.csv')

In [4]:
# Merge the training values and labels for a complete training dataset
train_data = pd.merge(train_values, train_labels, on='id')

In [7]:
# Display the first few rows of the dataset and its summary statistics
train_data.head(), train_data.describe(include='all')

(      id  amount_tsh date_recorded        funder  gps_height     installer  \
 0  69572      6000.0    2011-03-14         Roman        1390         Roman   
 1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
 2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
 3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
 4  19728         0.0    2011-07-13   Action In A           0       Artisan   
 
    longitude   latitude              wpt_name  num_private  ... water_quality  \
 0  34.938093  -9.856322                  none            0  ...          soft   
 1  34.698766  -2.147466              Zahanati            0  ...          soft   
 2  37.460664  -3.821329           Kwa Mahundi            0  ...          soft   
 3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...          soft   
 4  31.130847  -1.825359               Shuleni            0  ...          soft   
 
   quality_group      quantity

In [8]:
# Categorical columns with missing values
categorical_cols_missing = ['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name', 'permit']

In [9]:
# Fill missing values with the mode
for col in categorical_cols_missing:
    mode_value = train_data[col].mode()[0]
    train_data[col].fillna(mode_value, inplace=True)

In [10]:
# Encoding categorical variables using a label encoding strategy
# We will use label encoding instead of one-hot because of the high cardinality of some variables

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# List of categorical columns to encode
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('status_group')  # Exclude target variable

# Apply LabelEncoder to each categorical column
for col in categorical_cols:
    train_data[col] = label_encoder.fit_transform(train_data[col])

# Verify the encoding and missing value treatment
train_data.head(), train_data.isnull().sum()

(      id  amount_tsh  date_recorded  funder  gps_height  installer  longitude  \
 0  69572      6000.0             47    1368        1390       1518  34.938093   
 1   8776         0.0            309     469        1399        545  34.698766   
 2  34310        25.0            300     825         686       2048  37.460664   
 3  67743         0.0            272    1740         263       1852  38.486161   
 4  19728         0.0            104      20           0        119  31.130847   
 
     latitude  wpt_name  num_private  ...  water_quality  quality_group  \
 0  -9.856322     37398            0  ...              6              2   
 1  -2.147466     37194            0  ...              6              2   
 2  -3.821329     14572            0  ...              6              2   
 3 -11.155298     37284            0  ...              6              2   
 4  -1.825359     35528            0  ...              6              2   
 
    quantity  quantity_group  source  source_type  sou

In [11]:
from sklearn.model_selection import train_test_split

# Splitting the data into features and target
X = train_data.drop(['id', 'status_group'], axis=1) # 'id' is just an identifier and not a feature
y = train_data['status_group']

# Encoding the target variable
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Checking the shape of the datasets
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 39), (11880, 39), (47520,), (11880,))

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_clf.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = rf_clf.predict(X_val)

# Evaluate the model
classification_rep = classification_report(y_val, y_val_pred)

classification_rep


c:\Users\PeteD\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\PeteD\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\PeteD\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\PeteD\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future ver

'              precision    recall  f1-score   support\n\n           0       0.81      0.90      0.85      6457\n           1       0.56      0.35      0.43       851\n           2       0.85      0.79      0.82      4572\n\n    accuracy                           0.81     11880\n   macro avg       0.74      0.68      0.70     11880\nweighted avg       0.81      0.81      0.81     11880\n'